# Dependency Injection
----

In [38]:
from pprint import pprint

from dicfg.factory import ObjectFactory
from dicfg.reader import ConfigReader


class MyConfigReader(ConfigReader):
    NAME = "myconfig"

### To use dependency injection in our config we first need to setup a project

In [39]:
class ProjectComponent:
    def __init__(self, name):
        self.name = name


class MyProject:
    def __init__(self, project_component: ProjectComponent):
        self.project_component = project_component

### ObjectFactory can build objects with **object\*** key

In [68]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "object*": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
        },
    },
}

In [69]:
config = MyConfigReader.read(user_config)
pprint(config, sort_dicts=False)
objects = ObjectFactory.build(config["default"])

print("\n build objects: \n")

pprint(objects)

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'object*': '__main__.ProjectComponent',
                                   'name': 'my_project_component'}}}

 build objects: 

{'config_dict': {'sub_config': 'water'},
 'config_int': 1,
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_string': 'fire',
 'project_component': <__main__.ProjectComponent object at 0x7f84bcaab5e0>}


### Dependency Injection can be done by **object interpolation**

In [70]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "object*": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "object*": "__main__.MyProject",
                
                # object interpolation
                "project_component": "${project_component}",
            },
        },
    },
}

In [71]:
config = MyConfigReader.read(user_config)
pprint(config, sort_dicts=False)
objects = ObjectFactory.build(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)
print("\nproject component name:", objects["project"].project_component.name)

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'object*': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'object*': '__main__.MyProject',
                         'project_component': '${project_component}'}}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': <__main__.ProjectComponent object at 0x7f84bcab3dc0>,
 'project': <__main__.MyProject object at 0x7f84bcaaf0a0>}

project component name: my_project_component


### ObjectFactory interpolation of objects is done by **object reference**

In [72]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "object*": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "object*": "__main__.MyProject",
                
                # object interpolation by object reference
                "project_component": "${project_component}",
            },
            
            # object interpolation by object reference
            "my_interpolation_component": "${project_component}",
        },
    },
}

In [73]:
config = MyConfigReader.read(user_config)
pprint(config, sort_dicts=False)
objects = ObjectFactory.build(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)

print("\nreference       project_component:", objects["project_component"])
print("reference interpolation_component:", objects["my_interpolation_component"])

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'object*': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'object*': '__main__.MyProject',
                         'project_component': '${project_component}'},
             'my_interpolation_component': '${project_component}'}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': <__main__.ProjectComponent object at 0x7f84bd1e81f0>,
 'project': <__main__.MyProject object at 0x7f84bd1e83d0>,
 'my_interpolation_component': <__main__.ProjectComponent object at 0x7f84bd1e81f0>}

reference       project_component: <__main__.ProjectComponent object at 0x7f84bd1e81f0>
re

### ObjectFactory allows for **object attribute interpolation**

In [76]:
user_config = {
    "myconfig": {
        "default": {
            "project_component": {
                "object*": "__main__.ProjectComponent",
                "name": "my_project_component",
            },
            "project": {
                "object*": "__main__.MyProject",
                "project_component": "${project_component}",
            },
            "my_attribute_component": "${project_component}",
            
            # object attribute interpolation
            "my_attribute_component_name": "${project_component.name}",  
        },
    },
}

In [77]:
config = MyConfigReader.read(user_config)
pprint(config, sort_dicts=False)
objects = ObjectFactory.build(config["default"])
print("\n build objects: \n")
pprint(objects, sort_dicts=False)

print("\nmy_attribute_component_name:", objects["my_attribute_component_name"])

{'default': {'config_int': 1,
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'project_component': {'object*': '__main__.ProjectComponent',
                                   'name': 'my_project_component'},
             'project': {'object*': '__main__.MyProject',
                         'project_component': '${project_component}'},
             'my_attribute_component': '${project_component}',
             'my_attribute_component_name': '${project_component.name}'}}

 build objects: 

{'config_int': 1,
 'config_string': 'fire',
 'config_list': [1, 2, 3],
 'config_none': None,
 'config_dict': {'sub_config': 'water'},
 'project_component': <__main__.ProjectComponent object at 0x7f84bc6cd040>,
 'project': <__main__.MyProject object at 0x7f84bc6cd1f0>,
 'my_attribute_component': <__main__.ProjectComponent object at 0x7f84bc6cd040>,
 'my_attribute_component_n